In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
import tensorflow as tf
import pathlib
data = pathlib.Path("gdrive/My Drive/Data/program_machine/train")
test = pathlib.Path("gdrive/My Drive/Data/program_machine/test")

## ImageDataGenerator

- augmentation 하기 위해
- 같은 경로에서 가져와도 validation split과 subset을 이용해 분할 가능
- test는 스케일 적용만 함

In [156]:
imger = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,horizontal_flip=True,rotation_range=10,validation_split=0.15,width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,brightness_range=[0.9,1.1])
test_imger = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_temp = imger.flow_from_directory(data, batch_size=32, shuffle=True,subset='training')
valid_temp = imger.flow_from_directory(data, batch_size=32, shuffle=True,subset='validation')
test_temp = test_imger.flow_from_directory(test,batch_size=32,shuffle=False)

Found 1447 images belonging to 7 classes.
Found 251 images belonging to 7 classes.
Found 350 images belonging to 1 classes.


## Transfer Learning
- InceptionResNetV2 모델을 가져와 사용

In [15]:
m1 = tf.keras.applications.InceptionResNetV2(include_top=False, input_shape=(256,256,3))

219062272/219055592 [==============================] - 1s 0us/step


In [161]:
m1.trainable = False
global_average_pooling = tf.keras.layers.GlobalAveragePooling2D()
mid1 = tf.keras.layers.Dense(256)
mid2 = tf.keras.layers.Dense(64)
last = tf.keras.layers.Dense(7)

model = tf.keras.models.Sequential([
    m1,  global_average_pooling, mid, mid2, last
])

model.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 6, 6, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d_23  (None, 1536)              0         
_________________________________________________________________
dense_49 (Dense)             (None, 256)               393472    
_________________________________________________________________
dense_52 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_53 (Dense)             (None, 7)                 455       
Total params: 54,747,111
Trainable params: 410,375
Non-trainable params: 54,336,736
_________________________________________________________________


callback 적용

In [162]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

es = EarlyStopping(patience=10);
lr = ReduceLROnPlateau( factor=0.8, mode='min', patience=3, min_lr=0.00001)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             optimizer='adam',metrics=['accuracy'])


In [163]:
history = model.fit(train_temp, steps_per_epoch = train_temp.samples/32, validation_data = valid_temp, 
    validation_steps = valid_temp.samples/32, epochs = 100, callbacks=[es,lr])

Epoch 1/100
45/45 [==============================] - 42s 739ms/step - loss: 2.1621 - accuracy: 0.5574 - val_loss: 0.3064 - val_accuracy: 0.9283
Epoch 2/100
45/45 [==============================] - 31s 676ms/step - loss: 0.3545 - accuracy: 0.8926 - val_loss: 0.1802 - val_accuracy: 0.9482
Epoch 3/100
45/45 [==============================] - 31s 675ms/step - loss: 0.2531 - accuracy: 0.9152 - val_loss: 0.2291 - val_accuracy: 0.9522
Epoch 4/100
45/45 [==============================] - 31s 682ms/step - loss: 0.3228 - accuracy: 0.8941 - val_loss: 0.2279 - val_accuracy: 0.9442
Epoch 5/100
45/45 [==============================] - 31s 680ms/step - loss: 0.2679 - accuracy: 0.9088 - val_loss: 0.2349 - val_accuracy: 0.9442
Epoch 6/100
45/45 [==============================] - 31s 679ms/step - loss: 0.1977 - accuracy: 0.9397 - val_loss: 0.2312 - val_accuracy: 0.9363
Epoch 7/100
45/45 [==============================] - 31s 678ms/step - loss: 0.1701 - accuracy: 0.9513 - val_loss: 0.1967 - val_accuracy:

In [164]:
result = model.predict(test_temp)

## 결과

In [165]:
import pandas as pd
pred = pd.read_csv("gdrive/My Drive/Data/program_machine/test_answer_sample_.csv",index_col="Unnamed: 0")

In [166]:
pred['answer value'] = result.argmax(axis=1)
pred.to_csv("gdrive/My Drive/Data/program_machine/prediction2.csv")